In [1]:
import os
import glob
import re
import itertools
from collections import Counter


from nltk.tokenize import word_tokenize
import gensim
from gensim.models import KeyedVectors 
from gensim.scripts.glove2word2vec import glove2word2vec

import sys
import gc
import pandas as pd
import spacy
import time
import textacy
import re
import nltk
from collections import defaultdict
from itertools import takewhile
from operator import itemgetter
import re

from cytoolz import itertoolz
from numpy import nanmin, nanmax, zeros, NaN
import numpy as np
from spacy.parts_of_speech import CONJ, DET, NOUN, VERB
from spacy.tokens.span import Span as SpacySpan
from spacy.lang.en.stop_words import STOP_WORDS

import textacy
from textacy import spacy_utils, text_utils
from textacy.compat import unicode_
from textacy.spacy_utils import (normalized_str, get_main_verbs_of_sent,
                                 get_subjects_of_verb, get_objects_of_verb,
                                 get_span_for_compound_noun,
                                 get_span_for_verb_auxiliaries)
from textacy.constants import NUMERIC_NE_TYPES, REPORTING_VERBS
from wordcloud import WordCloud, ImageColorGenerator
import word2veckeras


pd.set_option('max_colwidth',-1)
pd.set_option('max_row',100)

a = time.time()

/home/wang/anaconda/lib/python3.6/site-packages/textacy/spacy_utils.py:25: DeprecationWarning: The `spacy_utils` module is deprecated and will be removed in v0.7.0.Use the `textacy.spacier` subpackage instead.
  action="once")


# 2. Data Preprocessing and opinion extraction

In [ ]:
#==================================   Import Data   =====================================

#import lexico
lexico_p = pd.read_csv('./positive-words.txt',  sep="\n", header = None)
lexico_p.columns = ['opinion_words']
lexico_p = lexico_p.loc[35:].reset_index(drop=True)
set_p = set(lexico_p['opinion_words'])
lexico_n = pd.read_csv('./negative-words.txt',  sep="\n", header = None)
lexico_n.columns = ['opinion_words']
lexico_n = lexico_n.loc[34:].reset_index(drop=True)
set_n = set(lexico_n['opinion_words'])
opinions = set_n | set_p
del lexico_n, lexico_p
gc.collect()

#import reviews data
path = '/home/wang/000Research_Fall2018/0_test/real'
appended_data = []
for filename in glob.glob(os.path.join(path, '*.csv')):
    temp  = pd.read_csv(filename)
    appended_data.append(temp)  
data = pd.concat(appended_data, ignore_index=True)
text = pd.DataFrame(columns=['text', 'tokens', 'pos'])

In [ ]:
##===================================   clean data   =======================================
#lot of cleaning here based on exploration
def replace(text):
    if type(text) is str:
        text = text.replace(' i ', ' I ')
        text = text.replace('<br />', ' ')
        text = text.replace('/', ' or ')
        text = text.replace('gonna', 'going to')
        text = text.replace('isn t', 'is not')
        text = text.replace('it s', 'it is')
        text = text.replace('like-it', 'like it')
        text = text.replace('love-it', 'love it')
        text = text.replace('aren t', 'are not')
        text = text.replace('can t', 'can not')
        text = text.replace('I like', 'I likes')
        text = text.replace('i like', 'i likes')
        text = text.replace('e like', 'we likes')
        text = text.replace('hey like', 'hey likes')
        text = text.replace('...', '.')
        text = text.replace('!!!', '!')
        if text.capitalize() == text:
            text = text.lower()   #transfer the Captatilize in 1st letter
        return text
    else:
        return 'NAN'
    
text['text'] = data['review_text'].apply(replace)

# 3. Extract opinions based on rules

In [13]:
#=====================================   word embedding   ==========================================


nlp = spacy.load('en_core_web_lg')

#split sentences also on , and ...
# def set_custom_boundaries(doc):
#     for token in doc[:-1]:
#         if token.text in ['...']:
#             doc[token.i+1].is_sent_start = True
#     return doc
# nlp.add_pipe(set_custom_boundaries, before='parser')

text['tokens'] = text['text'].apply(lambda x: nlp(x))

## RULES, EXAMPLE
## NOUN VERB ADJ ---> everything is ok
## NOUN VERB ADV ADJ ---> list is not correct
## NOUN VERB ADV ---> cars run fast
## NOUN VERB ADV( NOUN PRON VERB) ---> Clorets gum does
 
## ADJ PUNCT ADJ NOUN NOUN  ---> healthier, whole grain versions  
## ADJ NOUN NOUN --->  brown rice flour
## ADJ NOUN ADP NOUN ---> superior piece of equipment
## ADJ NOUN CCONJ NOUN ---> Great packing and speed
## VERB ADV VERB ---> do not buy
## PRON VERB VERB ---> I would recommend(过滤之后剩下recommend之类的金子, 筛掉沙子)
## VERB ADV ADV ADJ ADP ---> is not as good as
## ADV ADV ADJ ---> considerably more bland
## PRON ADV VERB ---> they absolutely love
## ((VERB PRON)|())?( ADV) ADJ PART VERB ---> (makes it/it is) very convenient to take out
## ADV ADV PRON VERB ---> how happily they munched on these
## VERB ADJ ADP( DET)?( ADJ)?( NOUN)?  ---> IS BETTER THAN 
## r'(VERB( PRON|( DET NOUN))? ADV)',   ---> I love it better
## VERB ADV ADJ ---> was so nasty
## NOUN', 'ADP', 'PRON -- > cats like it

   
def generate_phrase(tokens):
    print("="*90)
    res = []
    for sent in tokens.sents:
        if len(sent)<=3:  #etc, recommend! great!
            res.append(sent) 
        print("A: SENT------",sent)
        pos = [token.pos_ for token in sent]  #spaCy uses the dependency parse to determine sentence boundaries.
        pos_string = ' ' + ' '.join(pos)
#         print(pos_string)
#         p = re.compile(r'((NOUN VERB ADJ( PUNCT ADJ)?( CCONJ ADJ)?)|(NOUN VERB ADV ADJ( PUNCT ADJ)?( CCONJ ADJ)?( NOUN)*)|(NOUN VERB ADV( NOUN PRON VERB)?)|((ADV)?( ADJ PUNCT)?( ADJ CCONJ)? ADJ NOUN NOUN)|(ADJ NOUN ADP NOUN)|(ADJ NOUN CCONJ NOUN)|((ADV)? ADJ PART VERB( ADJ)? NOUN)|(VERB ADV VERB))|(PRON VERB( ADV)? VERB)|(VERB( ADV)? ADV ADJ ADP)|(ADV ADV ADJ)|(PRON ADV VERB( DET)?( NOUN)?)|(((VERB PRON)|())?( ADV) ADJ PART VERB)|( ADV ADV PRON VERB)' )

        regexps = [
            r'((ADJ)? ADJ NOUN)',
            r'((ADV )?ADJ (NOUN|PROPN) ADP (NOUN|PROPN))',
            r'((ADV )?ADJ (NOUN|PROPN) CCONJ (NOUN|PROPN))',
            r'((ADV )?ADJ( PUNCT ADJ)?( CCONJ ADJ)? (NOUN|PROPN))',
            r'((ADV )?(ADJ PUNCT)?( ADJ CCONJ)? ADJ (NOUN|PROPN) (NOUN|PROPN))',

            r'(((NOUN|PROPN)( VERB)?( ADV)? ADJ( PUNCT ADJ)?( CCONJ ADJ)?))', 
            r'((PRON VERB ( ADV)? ADJ( PUNCT ADJ)?( CCONJ ADJ)?))', 
            r'(ADJ PUNCT ADJ CCONJ ADJ)',
            
            r'((NOUN|PROPN)? VERB ADV ADJ( PUNCT ADJ)?( CCONJ ADJ)?((DET)? (NOUN|PROPN))?)',
            r'((NOUN|PROPN)? VERB ADV ADJ( PUNCT ADJ)?( CCONJ ADJ)? PROUN)',

            r'((NOUN|PROPN) VERB ADV( (NOUN|PROPN) PRON VERB)?)',
            r'(VERB ADV ADV)',
            r'(VERB ADV VERB)',
            r'((ADV)? ADJ PART VERB ADJ( (NOUN|PROPN))?)',
            r'((ADV)? ADJ PART VERB(DET)? (NOUN|PROPN)?)',
            r'((ADV)? ADJ PART VERB( CCONJ VERB))',
            r'(VERB( PRON) ADV)',
            r'(VERB( DET (NOUN|PROPN))( CCONJ( DET)? (NOUN|PROPN))?( ADV)?)',
            r'(ADV VERB DET( (NOUN|PROPN))?)',
            r'(VERB ADV ADJ)',
            r'(PRON VERB( ADV)? VERB)',
            r'(VERB( ADV)? ADV ADJ ADP)',
            r'(ADV ADV ADJ)',
            r'(PRON ADV VERB( DET)?( (NOUN|PROPN))?)', 
            r'(VERB PRON( ADV)? ADJ PART VERB)',
            r'(VERB PRON PART VERB ADJ)',
            r'( ADV ADV PRON VERB)', 
            r'(PRON VERB PRON (ADV)?)',
            r'((VERB)? ADJ ADP( DET)?( ADJ)?( (NOUN|PROPN))?)', 
            r'((NOUN|PROPN) ADP( DET (NOUN|PROPN)))',
            r'((NOUN|PROPN) ADP( PRON))',
            r'(ADV ADP DET (NOUN|PROPN))', 
            r'(VERB ADP( ADV)?( NUM)? (NOUN|PROPN))',
            r'((VERB)? VERB VERB( ADV)?)', 
            r'(VERB VERB PART VERB( ADJ|NOUN))',
            r'(NUM (NOUN|PROPN) PART)',
            r'(VERB ADP DET (NOUN|PROPN))',
            r'((NOUN|PROPN) ADV ADV)',
            r'(ADV VERB PRON)',
            r'((ADV)? ADJ PUNCT)',
            r'(ADJ)? NOUN VERB PRON', 
            r'(VERB( PUNCT) NOUN)',
            r'ADJ ADP NOUN',
            r'ADV NOUN PRON VERB',
            r'ADJ CCONJ VERB ADJ',
            
        ]
        
        for regexp in regexps:
            p = re.compile(regexp)
            if not p.search(pos_string):
                continue
            else:
                x = pos_string[p.search(pos_string).span()[0]:p.search(pos_string).span()[1]] 
                for m in re.finditer(x, pos_string):
                    res.append(sent[pos_string[0:m.start()].count(' ')-1:pos_string[0:m.end()].count(' ')])
                    print("D: RESULT------", sent[pos_string[0:m.start()].count(' ')-1:pos_string[0:m.end()].count(' ')], x, "\n")    
    print('\n', tokens, '\n')
    print(res)
    return res

def extract_lexico(span):
    """
    @input: a list containing spans format, candidates1 phrases
    @output: a list containing strs
    For each token in span ,
    Lemmatize and then find if any in opinion set 
    """
    res = set()
    for x in span:
        if len(x)!=0: 
            for tk in nlp(x.lemma_): 
                if tk.text in opinions:
                    temp = []
                    for w in x:
                        if not w.is_punct and w.lemma_ != '-PRON-':
                            temp.append(w.lemma_)
                    res.add(' '.join(temp))
                    #no dif between "I love" and "my girlfriend love"
                    break
    return list(res)

text['candidates1'] = text['tokens'].apply(generate_phrase)

#candidate1--------------------->candidates2
#Example: I love the flavor -----> Love the flavor
text['candidates2'] = text['candidates1'].apply(extract_lexico)
text = text[['text', 'candidates1', 'candidates2']]
text
# text.to_csv('./phrases.csv', index=False)

0        [nonsense about the k, disproportionate degree, no more nonsense, like the handle, digital monstrosity, buy to replace, other tiny improvement, the great idea, a strong brew setting, like about this machine, a strong brew, monstrosity that, tiny improvement, strong brew, kind of nice, the aforementioned lack, buy to, great idea, engineer digital monstrosity, aforementioned lack, of nice, more nonsense, of approve kick, absolutely love]                                                                                                                                                                                                                                   
1        [be not confident, not confident that, have be really good, fix the problem, water be cold, really good about, be really good about, the same problem, be cold, be not confident that, flaw in this coffee, same problem, be really good]                                                                                  